In [17]:
import pickle
from src.expressions_generator import ExpressionGenerator
from src.evaluate import make_model, Evaluator
import torch
from torch.utils.data import DataLoader
from src.dataset import DiffeqDataset
import sympy as sp
import numpy as np
from tqdm import tqdm

In [ ]:
def solution_to_sympy(prefix_ids):
    prefix_sol = [id2token[i] for i in prefix_ids]
    sympy_expr = gen.prefix_to_simpy(prefix_sol, has_y=True)
    return sympy_expr

In [ ]:
def infix_eq_to_prefix_id(infix_expr):
    sympy_expr = gen.infix_to_sympy(infix_expr)
    prefix_expr = gen.sympy_to_prefix(sympy_expr)
    prefix_expr = gen.replace_f_by_y(prefix_expr)
    prefix_id = [token2id['<s>']] + [token2id[tok] for tok in prefix_expr] + [token2id['</s>']]
    return sympy_expr, prefix_id

In [ ]:
def create_data_iterator(train, batch_size, path):
       
        dataset = DiffeqDataset(train, batch_size, path)
    
        return DataLoader(dataset,
                          batch_size=batch_size,
                          shuffle=False,
                          collate_fn=dataset.collate_fn
        )

## Опредление качества обученной модели

In [19]:
DATA_PATH = '/home/ilia/diff_eq_project/ode1_dataset/'
MODEL_PATH = '/home/ilia/diff_eq_project/ode_models/'

with open(DATA_PATH + '/vocabulary', "rb") as f:
    vocab = pickle.load(f)
    token2id = vocab['token2id']
    id2token = vocab['id2token']
    
VOCAB_SIZE = len(token2id)
N_LAYERS = 6
D_MODEL = 256
D_FF = 1024
N_HEADS = 8
DROPOUT = 0.1
PAD_IDX = token2id['<pad>']
BATCH_SIZE = 64
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cpu')

Загружаем лучшую обученную модель:

In [20]:
model = make_model(DEVICE, VOCAB_SIZE, PAD_IDX, N_LAYERS, 
               D_MODEL, D_FF, N_HEADS, DROPOUT).to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH + 'transformer_9.pt', map_location=torch.device('cpu')))
model.eval();

In [21]:
gen = ExpressionGenerator()
evaluator = Evaluator(model, DEVICE, vocab)

In [24]:
test_dataset = DiffeqDataset(train=False, batch_size=64, path=DATA_PATH)

In [25]:
x = gen.vars['x']
y = gen.vars['y']
z = gen.vars['z']
f = gen.functions['f']
c = gen.coef['C1']

Выберем некоторое количество уравнений из тестового датасета и посчитаем долю правильно решенных моделью уравнений:

In [29]:
check_sample = np.random.choice(len(test_dataset), size=500, replace=False)

In [30]:
count_right_solutions = 0
for i in tqdm(check_sample):
    prefix_eq_int = [int(id_token) for id_token in test_dataset[i][0]]
    prefix_sol_int = [int(id_token) for id_token in test_dataset[i][1]]
    beam_res, _, _= evaluator.beam_decode(prefix_eq_int, beam_size=5, alpha=1)
    sympy_sol = solution_to_sympy(prefix_sol_int[1:-1]).subs(f(x), y)
    transformer_sol = solution_to_sympy(beam_res).subs(f(x), y)
    transformer_sol_p = transformer_sol.subs([(x, 1.33), (y, 1.33), (c, 1.33)])
    sympy_sol_p = sympy_sol.subs([(x, 1.33), (y, 1.33), (c, 1.33)])
    if transformer_sol_p.equals(sympy_sol_p):
        count_right_solutions += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [10:56<00:00,  1.31s/it]


In [31]:
print("Accuracy: ",  count_right_solutions/500)

Accuracy:  0.75


## Пример решения дифференциального уравнения

Задаём ДУ, которое хотим решить:

In [38]:
infix_eq = 'Derivative(f(x), x) + sin(x)'

Переводим его в префиксную запись и токенизируем:

In [39]:
sympy_eq, id_eq = infix_eq_to_prefix_id(infix_eq)

"Решаем" ДУ с помощью модели:

In [46]:
beam_best_res, hyps, hyps_probs = evaluator.beam_decode(id_eq, beam_size=5, alpha=0)

Исходное уравнение:

In [41]:
sympy_eq

sin(x) + Derivative(f(x), x)

Полученное моделью решение:

In [47]:
sol = solution_to_sympy(beam_best_res)
sol

C1 + f(x) - cos(x)